In [1]:
import pickle
import sys
import os
import plotly.graph_objects as go

In [2]:
current_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_dir, '..')))
from utils.scripts.functions.main_functions import forecast_next_24_hours_output_flow_rate

In [3]:
import pandas as pd
df0 = pd.read_parquet("../data/curated_datasets/water_consumption_curated.parquet")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', None)

In [22]:
def select_first_and_middle(group):
    rows = group.iloc[::7]
    return rows


In [5]:
forecaster = pickle.load(open("../models/forecaster.pkl", 'rb'))
forecaster_with_weather = pickle.load(open("../models/forecaster_with_weather.pkl", 'rb'))
input_forecaster = pickle.load(open("../models/input_flow_forecaster.pkl", 'rb'))

In [6]:
output_flow_24_hour_forecast_question_2_dataset, df = forecast_next_24_hours_output_flow_rate(df0, forecaster, input_forecaster, 2023, 9, 16, 14, 15)

In [23]:
real_data = df[df['forecasted'] == False].groupby('hour', group_keys=False).apply(select_first_and_middle)
forecasted_data = df[df['forecasted'] == True].groupby('hour', group_keys=False).apply(select_first_and_middle)
real_data_complete = pd.concat([real_data, forecasted_data]).reset_index(drop=True)

/tmp/ipykernel_46005/2877401902.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

/tmp/ipykernel_46005/2877401902.py:2: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [31]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=real_data_complete.index[real_data_complete['forecasted'] == False],y=real_data_complete['output_flow_rate'][real_data_complete['forecasted'] == False],mode='lines+markers',name='Ultimas 24 horas',line=dict(color='blue')))
fig.add_trace(go.Scatter(x=real_data_complete.index[real_data_complete['forecasted'] == True],y=real_data_complete['output_flow_rate'][real_data_complete['forecasted'] == True],mode='lines+markers',name='Previsao das proximas 24 horas',line=dict(color='red')))
fig.update_layout(title={'text': 'Previsao da Saida de Agua', 'x': 0.5, 'xanchor': 'center'},xaxis_title='',yaxis_title='Saida de Agua (L/s)',legend_title='Type')
fig.update_traces(marker=dict(symbol='circle'))
fig.show()